In [ ]:
import numpy as np
import pandas as pd
import time
import re
import math
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn import metrics 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable
from torch.nn.modules import Module
from torch.utils.data import Dataset, DataLoader

from torch_geometric.data import Data
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils import add_remaining_self_loops, add_self_loops
from torch_geometric.utils import to_undirected
from torch_geometric.nn import GCNConv, SAGEConv,GAE, VGAE
from pcgrad import PCGrad
import gc
from tqdm import tqdm

In [ ]:
data_path = './AD_project/pre_process_data/'
exp_id='exp1'
le=pickle.load(open(data_path+'AD_LabelEncoder_'+exp_id+'_kh.pkl', 'rb'))
edge_index=pickle.load(open(data_path+'AD_edge_index_'+exp_id+'_kh.pkl','rb'))
edge_index=edge_index[edge_index.type!='AD_related_inf']
node_feature_np=pickle.load(open(data_path+'AD_node_feature_'+exp_id+'_kh.pkl','rb'))
node_class=pickle.load(open(data_path+'AD_gene_node_class_'+exp_id+'_kh.pkl','rb'))
node_class_mask=pickle.load(open(data_path+'gene_mask_'+exp_id+'_kh.pkl','rb'))
AD_gene_index=pickle.load(open(data_path+'ad_gene_index_'+exp_id+'_kh.pkl','rb'))
non_AD_gene_index=pickle.load(open(data_path+'non_ad_gene_index_'+exp_id+'_kh.pkl','rb'))

In [ ]:
z_np=pickle.load(open('./AD_project/pre_process_data/drug_comb_node_embedding_comb_with_pretrain_kh.pkl', 'rb'))

In [ ]:
types=np.array([item.split('_')[0] for item in le.classes_ ])
name=np.array([item.split('_')[1] for item in le.classes_ ])
drug_df=pd.DataFrame(types,columns=['type'])
drug_df['name']=name
drug_df=drug_df[drug_df.type=='drug']
drug_df['name']=[i.title() for i in drug_df['name'] ]

In [ ]:
data_path='./AD_project/data' 
CTDdruginf=readpkl(data_path+'/CTD/CTD_chemicals.pkl.gz')
CTDdruginf=CTDdruginf[['# ChemicalName','Synonyms']]
allsym=[]
allctddrug=[]
for i in range(len(CTDdruginf)):
    temp=str(CTDdruginf.Synonyms.values[i]).split('|')
    temp.append(str(CTDdruginf['# ChemicalName'].values[i]))
    temp=[str(word).title() for word in temp ]
    for j in temp:
        allsym.append(j)
        allctddrug.append(CTDdruginf['# ChemicalName'].values[i])
CTDdruginf=pd.DataFrame(allctddrug,columns=['# ChemicalName'])
CTDdruginf['sym']=allsym
CTDdruginf['# ChemicalName']=[i.title() for i in CTDdruginf['# ChemicalName']]

In [ ]:
EHR=pd.read_csv('./AD_project/data/drug_effect_1drug_scan_adrd_v2.1_best_20210627.csv')
EHR=EHR[EHR['t-stats (ATT)']>0]
EHR=EHR[EHR['pvalue (ATT)']<0.001]
EHR['drug_name']=[str(i).title() for i in EHR['drug_name']]
EHR=EHR[['drug_name']]
EHR.columns=['sym']
EHR=EHR.merge(CTDdruginf)
EHR.columns=['sym','EHR_drugs']
EHR_drug=list(EHR['EHR_drugs'].values)
EHR_drug=[i.upper() for i in EHR_drug]
print(len(EHR_drug))
gesa=pd.read_csv('./AD_project/pre_process_data/gesa_ad_kh.csv')
gesa['Gesa_drug']=[str(i).title() for i in gesa['Gesa_drug']]
gesa.columns=['sym']
gesa=gesa.merge(CTDdruginf)
gesa.columns=['sym','gesa_drugs']
gesa_drug=list(gesa['gesa_drugs'].values)
gesa_drug=[i.upper() for i in gesa_drug]
print(len(gesa_drug))
pre_trial=pd.read_csv('./AD_project/pre_process_data/AD_pre_drug_kh.csv')
pre_trial['AD_pos_drugs']=[str(i).title() for i in pre_trial['AD_pos_drugs']]
pre_trial=pre_trial[['AD_pos_drugs']]
pre_trial.columns=['sym']
pre_trial=pre_trial.merge(CTDdruginf)
pre_trial.columns=['sym','pre_trial_drugs']
pre_trial_drug=list(pre_trial['pre_trial_drugs'].values)
pre_trial_drug=[i.upper() for i in pre_trial_drug]
print(len(pre_trial_drug))
trials_drug=list(pd.read_csv('./AD_project/pre_process_data/trial_ctd.csv')['CTD_name'].values)
trials_drug_1=list(pd.read_csv('./AD_project/pre_process_data/alzforum_mesh.csv')['ctdname'].values)
trials_drug=[i.upper() for i in trials_drug]
trials_drug=list(set(trials_drug+trials_drug_1))
print(len(trials_drug))
all_drugs=list(set(trials_drug+EHR_drug+gesa_drug+pre_trial_drug))
trial_drug_labels=[1 if drug.split('_')[1] in trials_drug else 0 for drug in le.classes_[types=='drug'] ]
gesa_drug_labels=[1 if drug.split('_')[1] in gesa_drug else 0 for drug in le.classes_[types=='drug'] ]
EHR_drug_labels=[1 if drug.split('_')[1] in EHR_drug else 0 for drug in le.classes_[types=='drug'] ]
pre_trial_labels=[1 if drug.split('_')[1] in pre_trial_drug else 0 for drug in le.classes_[types=='drug'] ]
all_drug_labels=[1 if drug.split('_')[1] in all_drugs else 0 for drug in le.classes_[types=='drug'] ]

In [ ]:
print(np.count_nonzero(EHR_drug_labels))
print(np.count_nonzero(trial_drug_labels))
print(np.count_nonzero(gesa_drug_labels))
print(np.count_nonzero(pre_trial_labels))
print(np.count_nonzero(all_drug_labels))

In [ ]:
drug_df['trial']=trial_drug_labels
drug_df['gesa']=gesa_drug_labels
drug_df['EHR']=EHR_drug_labels
drug_df['pre_trial']=pre_trial_labels
drug_df['all_label']=all_drug_labels

In [ ]:
class MLT_Classifier(nn.Module):
    def __init__(self,embedding_dim=embedding_size):
        super(MLT_Classifier_1, self).__init__()
        self.fc1=nn.Linear(embedding_dim,embedding_dim)
        self.fc2=nn.Linear(embedding_dim,1)
        self.fc3=nn.Linear(embedding_dim,1)
        self.fc4=nn.Linear(embedding_dim,1)
        self.fc5=nn.Linear(embedding_dim,1)
        self.fc6=nn.Linear(embedding_dim,1)
        self.bn=nn.BatchNorm1d(embedding_dim)
        
    def forward(self, x):
        residual1 = x
        x= self.bn(F.dropout(F.relu(self.fc1(x)),training=self.training))
        x += residual1  
        trial_out=self.fc2(x)
        ehr_out=self.fc3(x)
        gesa_out=self.fc4(x)
        pre_trial_out=self.fc5(x)
        final_out=self.fc6(x)
        
        return trial_out,ehr_out,gesa_out,pre_trial_out,final_out

In [ ]:
from torch.utils.data import BatchSampler, WeightedRandomSampler
class BPRLoss(nn.Module):
    def __init__(self, num_neg_samples):
        super(BPRLoss, self).__init__()
        self.num_neg_samples=num_neg_samples
    
    def forward(self, output, label):
        positive_output=output[label==1]
        #print(positive_output)
        negative_output=output[label!=1]
        #print(negative_output)        
        negative_sampler=WeightedRandomSampler(negative_output-min(negative_output), num_samples=self.num_neg_samples*len(positive_output),replacement=True)
        negative_sample_output=negative_output[torch.tensor(list(BatchSampler(negative_sampler, batch_size=len(positive_output),drop_last=True)),dtype=torch.long).t()]
        return -(positive_output.view(-1,1)-negative_sample_output).sigmoid().log().mean()


In [ ]:
def train_fu(model,data,num_neg_samples):
    learnable_params=model.parameters()
    optimizer = PCGrad(torch.optim.Adam(learnable_params)) 
    model.train()
    optimizer.zero_grad()
    trial_out,ehr_out,gesa_out,pre_trial_out,final_out=model(data['x_train'])
    #print(trial_out)
    #print(data['trial_y_train'])
    
    criterion_trial=BPRLoss(num_neg_samples=num_neg_samples)
    criterion_ehr=BPRLoss(num_neg_samples=num_neg_samples)
    criterion_gesa=BPRLoss(num_neg_samples=num_neg_samples)
    criterion_pre_trial=BPRLoss(num_neg_samples=num_neg_samples)
    criterion_all_drug=BPRLoss(num_neg_samples=num_neg_samples)
    
    trial_loss=criterion_trial(trial_out.squeeze(),data['trial_y_train'])
    ehr_loss=criterion_ehr(ehr_out.squeeze(),data['ehr_y_train'])
    gesa_loss=criterion_gesa(gesa_out.squeeze(),data['gesa_y_train'])
    pre_trial_loss=criterion_pre_trial(pre_trial_out.squeeze(),data['pre_trial_y_train'])
    all_drug_loss=criterion_all_drug(final_out.squeeze(),data['all_drug_y_train'])
    #model_kl_loss=kl_divergence_from_nn(model)

    loss_list= [trial_loss,ehr_loss,gesa_loss,pre_trial_loss,all_drug_loss]#,model_kl_loss]
    optimizer.pc_backward(loss_list)
    optimizer.step()

In [ ]:
def eval_fu(model,data):
    model.eval()
    with torch.no_grad():
        trial_out,ehr_out,gesa_out,pre_trial_out,final_out=model(data['x_eval'])
        
        trial_out=trial_out.sigmoid().squeeze().detach().cpu().numpy()
        ehr_out=ehr_out.sigmoid().squeeze().detach().cpu().numpy()
        gesa_out=gesa_out.sigmoid().squeeze().detach().cpu().numpy()
        pre_trial_out=pre_trial_out.sigmoid().squeeze().detach().cpu().numpy()
        final_out=final_out.sigmoid().squeeze().detach().cpu().numpy()
        
        data['trial_y_eval']=data['trial_y_eval'].squeeze().detach().cpu().numpy()
        data['ehr_y_eval']=data['ehr_y_eval'].squeeze().detach().cpu().numpy()
        data['gesa_y_eval']=data['gesa_y_eval'].squeeze().detach().cpu().numpy()
        data['pre_trial_y_eval']=data['pre_trial_y_eval'].squeeze().detach().cpu().numpy()
        data['all_drug_y_eval']=data['all_drug_y_eval'].squeeze().detach().cpu().numpy()
        
        
        trial_auc=metrics.roc_auc_score(data['trial_y_eval'], trial_out)
        ehr_auc=metrics.roc_auc_score(data['ehr_y_eval'], ehr_out)
        gesa_auc=metrics.roc_auc_score(data['gesa_y_eval'], gesa_out)
        pre_trial_auc=metrics.roc_auc_score(data['pre_trial_y_eval'], pre_trial_out)
        all_auc=metrics.roc_auc_score(data['all_drug_y_eval'], final_out)
        
        trial_apc=metrics.average_precision_score(data['trial_y_eval'], trial_out)
        ehr_apc=metrics.average_precision_score(data['ehr_y_eval'], ehr_out)
        gesa_apc=metrics.average_precision_score(data['gesa_y_eval'], gesa_out)
        pre_trial_apc=metrics.average_precision_score(data['pre_trial_y_eval'], pre_trial_out)
        all_apc=metrics.average_precision_score(data['all_drug_y_eval'], final_out)
        
        
    return [trial_auc,ehr_auc,gesa_auc,pre_trial_auc,all_auc],[trial_apc,ehr_apc,gesa_apc,pre_trial_apc,all_apc]


In [ ]:
def test_fu(model,data):
    model.eval()
    with torch.no_grad():
        trial_out,ehr_out,gesa_out,pre_trial_out,final_out=model(data['x_full'])
        
        trial_out=trial_out.sigmoid().squeeze().detach().cpu().numpy()
        ehr_out=ehr_out.sigmoid().squeeze().detach().cpu().numpy()
        gesa_out=gesa_out.sigmoid().squeeze().detach().cpu().numpy()
        pre_trial_out=pre_trial_out.sigmoid().squeeze().detach().cpu().numpy()
        final_out=final_out.sigmoid().squeeze().detach().cpu().numpy()
        
        data['trial_y_full']=data['trial_y_full'].squeeze().detach().cpu().numpy()
        data['ehr_y_full']=data['ehr_y_full'].squeeze().detach().cpu().numpy()
        data['gesa_y_full']=data['gesa_y_full'].squeeze().detach().cpu().numpy()
        data['pre_trial_y_full']=data['pre_trial_y_full'].squeeze().detach().cpu().numpy()
        data['all_drug_y_full']=data['all_drug_y_full'].squeeze().detach().cpu().numpy()
        
        
        trial_auc=metrics.roc_auc_score(data['trial_y_full'], trial_out)
        ehr_auc=metrics.roc_auc_score(data['ehr_y_full'], ehr_out)
        gesa_auc=metrics.roc_auc_score(data['gesa_y_full'], gesa_out)
        pre_trial_auc=metrics.roc_auc_score(data['pre_trial_y_full'], pre_trial_out)
        all_auc=metrics.roc_auc_score(data['all_drug_y_full'], final_out)
        
        trial_apc=metrics.average_precision_score(data['trial_y_full'], trial_out)
        ehr_apc=metrics.average_precision_score(data['ehr_y_full'], ehr_out)
        gesa_apc=metrics.average_precision_score(data['gesa_y_full'], gesa_out)
        pre_trial_apc=metrics.average_precision_score(data['pre_trial_y_full'], pre_trial_out)
        all_apc=metrics.average_precision_score(data['all_drug_y_full'], final_out)
        
    return [trial_auc,ehr_auc,gesa_auc,pre_trial_auc,all_auc],[trial_apc,ehr_apc,gesa_apc,pre_trial_apc,all_apc],final_out


In [ ]:
best_auprc=0

drug_labels_list=[i for i in range(z_np[types=='drug'].shape[0])]
clf=MLT_Classifier_1(128)
eval_auc=[]
eval_apc=[]
#train_indice=drug_labels_list[0:int(round(len(trial_drug_labels)*0.5,0))]
#eval_indice=drug_labels_list[int(round(len(trial_drug_labels)*0.5,0)):]
for epoch in tqdm(range(120)):
    MTL_data={}
    np.random.shuffle(drug_labels_list)
    train_indice=drug_labels_list[0:int(round(len(drug_labels_list)*0.8,0))]
    eval_indice=drug_labels_list[int(round(len(drug_labels_list)*0.8,0)):]
    x_train=z_np[types=='drug'][train_indice]
    y_trial_train=np.array(trial_drug_labels)[train_indice]
    y_gesa_train=np.array(gesa_drug_labels)[train_indice]
    y_ehr_train=np.array(EHR_drug_labels)[train_indice]
    y_pre_trial_train=np.array(pre_trial_labels)[train_indice]
    y_all_drug_train=np.array(all_drug_labels)[train_indice]
    MTL_data['x_train']=Variable(torch.tensor(x_train,dtype=torch.float))
    MTL_data['trial_y_train']=Variable(torch.tensor(y_trial_train,dtype=torch.float))
    MTL_data['ehr_y_train']=Variable(torch.tensor(y_gesa_train,dtype=torch.float))
    MTL_data['gesa_y_train']=Variable(torch.tensor(y_ehr_train,dtype=torch.float))
    MTL_data['pre_trial_y_train']=Variable(torch.tensor(y_pre_trial_train,dtype=torch.float))
    MTL_data['all_drug_y_train']=Variable(torch.tensor(y_all_drug_train,dtype=torch.float))
    
    
    x_eval=z_np[types=='drug'][eval_indice]
    y_trial_eval=np.array(trial_drug_labels)[eval_indice]
    y_gesa_eval=np.array(gesa_drug_labels)[eval_indice]
    y_ehr_eval=np.array(EHR_drug_labels)[eval_indice]
    y_pre_trial_eval=np.array(pre_trial_labels)[eval_indice]
    y_all_drug_eval=np.array(all_drug_labels)[eval_indice]
    MTL_data['x_eval']=Variable(torch.tensor(x_eval,dtype=torch.float))
    MTL_data['trial_y_eval']=Variable(torch.tensor(y_trial_eval,dtype=torch.float))
    MTL_data['ehr_y_eval']=Variable(torch.tensor(y_gesa_eval,dtype=torch.float))
    MTL_data['gesa_y_eval']=Variable(torch.tensor(y_ehr_eval,dtype=torch.float))
    MTL_data['pre_trial_y_eval']=Variable(torch.tensor(y_pre_trial_eval,dtype=torch.float))
    MTL_data['all_drug_y_eval']=Variable(torch.tensor(y_all_drug_eval,dtype=torch.float))
    
    
    train_fu(clf,MTL_data,200)
    auc,apc=eval_fu(clf,MTL_data)
    eval_auc.append(auc)
    eval_apc.append(apc)
    if auc[4]>best_auprc:
        best_auproc=auc[4]
        print(auc[4])
        torch.save(clf, './AD_project/pre_process_data/nn_clf-temp_kh_temp.pt')

In [ ]:
MTL_data={}
MTL_data['x_full']=Variable(torch.tensor(z_np[types=='drug'],dtype=torch.float))
MTL_data['trial_y_full']=Variable(torch.tensor(trial_drug_labels,dtype=torch.float))
MTL_data['gesa_y_full']=Variable(torch.tensor(gesa_drug_labels,dtype=torch.float))
MTL_data['ehr_y_full']=Variable(torch.tensor(EHR_drug_labels,dtype=torch.float))
MTL_data['pre_trial_y_full']=Variable(torch.tensor(pre_trial_labels,dtype=torch.float))
MTL_data['all_drug_y_full']=Variable(torch.tensor(all_drug_labels,dtype=torch.float))

In [ ]:
clf.load_state_dict(torch.load('./AD_project/pre_process_data/nn_clf-temp_kh_temp.pt').state_dict())
auc,apc,final_predict=test_fu(clf,MTL_data)
print('AUROC_trial :{:.4f} AUROC_gesa: {:.4f}, AUROC_ehr: {:.4f},AUROC_pre_trial: {:.4f},AUROC_all_drug: {:.4f}'.format(auc[0],auc[1],auc[2],auc[3],auc[4]))
print('AUPRC_trial :{:.4f} AUPRC_gesa: {:.4f}, AUPRC_ehr: {:.4f},AUPRC_pre_trial: {:.4f},AUPRC_all_drug: {:.4f}'.format(apc[0],apc[1],apc[2],apc[3],apc[4]))



In [ ]:
drug_df['final_prob']=final_predict
drug_df=drug_df.sort_values(by=['final_prob'],ascending=False)

In [ ]:
all_pr=[]
for i in tqdm(range(len(drug_df))):
    temp=drug_df[drug_df.ind<=i]
    temp['pred_label']=1
    all_pr.append(metrics.precision_score(temp['all_label'].values,temp['pred_label'].values))
drug_df['precision_score']=all_pr

In [ ]:
drug_df.to_csv('./AD_project/pre_process_data/top300_drugs_kh_new_oct_11.csv',index=False)